In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import dask

In [2]:
datapath = "/mnt/toparctic-ns9869k-ns9252k-blomchannel/BLOM_channel_new05_mix1/"

In [3]:
dx = 2e3             # [m]
dy = 2e3             # [m]
rho = 1e3            # stemmer dette?
dt = 1*24*60*60      # [s]

In [4]:
xchunk = -1
ychunk = -1
sigmachunk = -1
depthchunk= -1
timechunk = 30

In [5]:
# read data for one month, for testing on small data set
ds = xr.open_mfdataset(datapath+"*hd_2027.01.nc", 
                       chunks={"x":xchunk, "y":ychunk, 
                               "sigma":sigmachunk, "depth":depthchunk,
                               "time":timechunk}
                      ).sel(depth=slice(0, 2300))
pp = xr.Dataset()

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


In [6]:
ds

<xarray.Dataset>
Dimensions:     (depth: 52, bounds: 2, time: 30, y: 512, x: 208, sigma: 53,
                 lat: 510, region: 3, section: 1)
Coordinates:
  * depth       (depth) float64 0.0 5.0 10.0 15.0 ... 1.875e+03 2e+03 2.25e+03
  * time        (time) object 2027-02-01 12:00:00 ... 2027-02-30 12:00:00
  * sigma       (sigma) float64 33.21 33.21 33.54 33.84 ... 35.79 35.79 35.8
  * lat         (lat) float64 2.0 3.0 4.0 5.0 6.0 ... 508.0 509.0 510.0 511.0
  * region      (region) |S50 b'global                                       ...
  * section     (section) |S50 b'middle_section                              ...
Dimensions without coordinates: bounds, y, x
Data variables: (12/37)
    depth_bnds  (depth, bounds) float64 dask.array<chunksize=(52, 2), meta=np.ndarray>
    sigmx       (time, y, x) float32 dask.array<chunksize=(30, 512, 208), meta=np.ndarray>
    ubaro       (time, y, x) float32 dask.array<chunksize=(30, 512, 208), meta=np.ndarray>
    vbaro       (time, y, x) float32 dask.array<chunksize=(30, 512, 208), meta=np.ndarray>
    pbot        (time, y, x) float32 dask.array<chunksize=(30, 512, 208), meta=np.ndarray>
    sealv       (time, y, x) float32 dask.array<chunksize=(30, 512, 208), meta=np.ndarray>
    ...          ...
    bfsqlvl     (time, depth, y, x) float32 dask.array<chunksize=(30, 52, 512, 208), meta=np.ndarray>
    pvlvl       (time, depth, y, x) float32 dask.array<chunksize=(30, 52, 512, 208), meta=np.ndarray>
    mmflxl      (time, region, sigma, lat) float64 dask.array<chunksize=(30, 3, 53, 510), meta=np.ndarray>
    mmflxd      (time, region, depth, lat) float64 dask.array<chunksize=(30, 3, 52, 510), meta=np.ndarray>
    mhflx       (time, region, lat) float64 dask.array<chunksize=(30, 3, 510), meta=np.ndarray>
    voltr       (time, section) float64 dask.array<chunksize=(30, 1), meta=np.ndarray>

In [7]:
# shift u and v to cell center
ni = len(ds.x)
nj = len(ds.y)

u = ds.uvellvl
v = ds.vvellvl

uc = xr.concat([u.isel(x=slice(0,ni-1)), u.isel(x=slice(1,ni))], dim="temp").mean(dim="temp")
ucend = xr.concat([u.isel(x=0), u.isel(x=-1)], dim="temp").mean(dim="temp")
uc = xr.concat([uc, ucend], dim="x")
pp["uc"] = uc#.chunk({"x":-1, "y":-1, "sigma":-1, "time":10})


vc = xr.concat([v.isel(y=slice(0,nj-1)), v.isel(y=slice(1,nj))], dim="temp").mean(dim="temp")
vcend = v.isel(y=-1)*np.nan
vc = xr.concat([vc, vcend], dim="y")

pp["vc"] = vc.chunk({"x":xchunk, "y":ychunk, "depth":depthchunk, "time":timechunk})

In [8]:
# spesify coordinate values, usefull for differentiating
pp["x"] = dx*np.arange(0,ni)
pp["y"] = dy*np.arange(0,nj)
pp

<xarray.Dataset>
Dimensions:  (depth: 52, time: 30, y: 512, x: 208)
Coordinates:
  * depth    (depth) float64 0.0 5.0 10.0 15.0 ... 1.875e+03 2e+03 2.25e+03
  * time     (time) object 2027-02-01 12:00:00 ... 2027-02-30 12:00:00
  * x        (x) float64 0.0 2e+03 4e+03 6e+03 ... 4.1e+05 4.12e+05 4.14e+05
  * y        (y) float64 0.0 2e+03 4e+03 6e+03 ... 1.018e+06 1.02e+06 1.022e+06
Data variables:
    uc       (time, depth, y, x) float32 dask.array<chunksize=(30, 52, 512, 207), meta=np.ndarray>
    vc       (time, depth, y, x) float32 dask.array<chunksize=(30, 52, 512, 208), meta=np.ndarray>

In [9]:
# pad u in reentranse direction
upad = xr.concat([pp.uc.isel(x=-1), pp.uc, pp.uc.isel(x=0)], dim="x").chunk({"x":-1, "y":-1})
upad["x"] = dx*np.arange(-1,len(pp.x)+1)

# Calculate derivative of u and v using central difference    
dudx = upad.differentiate("x").isel(x=slice(1,-1))
dvdy = pp.vc.differentiate("y")

In [10]:
# use continuity to calculate derivative of vertical velocity w
dwdz = -(dudx+dvdy)

# calculate dz
dz = (ds.depth_bnds.isel(bounds=1)-ds.depth_bnds.isel(bounds=0))
dz.compute()

<xarray.DataArray 'depth_bnds' (depth: 52)>
array([  2.5,   5. ,   5. ,   5. ,   5. ,   5. ,   7.5,  10. ,  11.2,
        12.6,  12.4,  12.6,  12.4,  12.6,  12.4,  12.6,  18.7,  25. ,
        25. ,  25. ,  25. ,  25. ,  37.5,  50. ,  50. ,  50. ,  50. ,
        50. ,  50. ,  50. ,  50. ,  50. ,  50. ,  50. ,  50. ,  50. ,
        50. ,  50. ,  50. ,  50. ,  50. ,  50. ,  50. ,  50. ,  50. ,
        50. ,  87.5, 125. , 125. , 125. , 187.5, 250. ])
Coordinates:
  * depth    (depth) float64 0.0 5.0 10.0 15.0 ... 1.875e+03 2e+03 2.25e+03

In [11]:
# calculate vertical velocities at surface due to changes in sea surface elevation
ws = ds.sealv.differentiate("time", datetime_unit="s")

In [12]:
# integrate dwdz from surface and down
wc1 = ws+(-dwdz*dz).cumsum("depth")
pp["wc1"] = wc1

In [16]:
# calculate vertical velocities from mass fluxes
A = dx*dy
wflx = ds.wflxlvl
wc2 = wflx/(rho*A)
pp["wc2"] = wc2

In [19]:
# calculate mean over time
pp_mean = pp.mean("time")
pp_mean.uc.attrs = {"long_name" : "Horizontal time-mean x-velocity at grid center", "units" : "m s-1"}
pp_mean.vc.attrs = {"long_name" : "Horizontal time-mean y-velocity at grid center", "units" : "m s-1"}
pp_mean.wc1.attrs = {"long_name" : "Vertical time-mean velocity at grid center calculated from continuity and change in sea surface elevation", "units" : "m s-1"}
pp_mean.wc2.attrs = {"long_name" : "Vertical time-mean velocity at grid center calculated from vertical mass flux", "units" : "m s-1"}

In [20]:
pp_mean.compute()

<xarray.Dataset>
Dimensions:  (depth: 52, y: 512, x: 208)
Coordinates:
  * depth    (depth) float64 0.0 5.0 10.0 15.0 ... 1.875e+03 2e+03 2.25e+03
  * x        (x) float64 0.0 2e+03 4e+03 6e+03 ... 4.1e+05 4.12e+05 4.14e+05
  * y        (y) float64 0.0 2e+03 4e+03 6e+03 ... 1.018e+06 1.02e+06 1.022e+06
Data variables:
    uc       (depth, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    vc       (depth, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    wc1      (y, x, depth) float64 nan nan nan nan nan ... nan nan nan nan nan
    wc2      (depth, y, x) float32 nan nan nan nan nan ... nan nan nan nan nan